**Summmary:**

In [10]:
import pandas as pd
import gzip
import json
wyoming_ratings_only = pd.read_csv("rating-Wyoming.csv")

def parse(path):
  g = open(path, 'r')
  for l in g:
    yield json.loads(l)



wyoming_reviews = pd.DataFrame(parse("review-Wyoming_10.json"))


In [11]:
wyoming_metadata = pd.DataFrame(parse("meta-other.json"))

In [12]:
wyoming_reviews #contains business_id, user_id, rating, timestamp


,user_id,name,time,rating,text,pics,resp,gmap_id
0,111435346941066248277,Jenn Thomas,1602893531994,5,"When knowledge is key and kindness matters, Ni...",None,None,0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
1,103613723635264236030,Michelle Barta,1575991509006,5,The entire team is outstanding! They are profe...,None,"{'time': 1580484156696, 'text': 'We are lucky ...",0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
2,113556685435236755030,Bethany Baldes,1626981484302,5,None,None,None,0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
3,102235352482458236558,nichole vogt,1583381337082,5,None,None,None,0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
4,114021734869233847378,Kristi Friday,1571112557173,4,None,None,"{'time': 1571246498430, 'text': 'Thanks for th...",0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
...,...,...,...,...,...,...,...,...
427803,107899148903686037545,Carlene Calabaza,1614741323428,5,None,None,None,0x53349429523d615b:0x2a66a9272032e8b4
427804,117387828996525813955,Gizmo,1561931911835,5,None,None,None,0x53349429523d615b:0x2a66a9272032e8b4
427805,107887136514890657569,Jan Soucek,1501029366785,5,None,None,None,0x53349429523d615b:0x2a66a9272032e8b4
427806,104765216668868688471,Kyle Leatherow,1565466860108,3,None,None,None,0x53349429523d615b:0x2a66a9272032e8b4


In [13]:
wyoming_ratings_only

,business,user,rating,timestamp
0,0x875901dbacdf1fb7:0x5e0aea8e0909ba5c,115646077423418438821,5,1570739784825
1,0x875901dbacdf1fb7:0x5e0aea8e0909ba5c,105099928136753216602,5,1505421891766
2,0x875901dbacdf1fb7:0x5e0aea8e0909ba5c,115843664516717640522,5,1550017414366
3,0x875901dbacdf1fb7:0x5e0aea8e0909ba5c,103613723635264236030,5,1504928708627
4,0x875901dbacdf1fb7:0x5e0aea8e0909ba5c,113597382441988504332,5,1598022469876
...,...,...,...,...
1124366,0x53349429523d615b:0x2a66a9272032e8b4,104765216668868688471,3,1565466860108
1124367,0x53349429523d615b:0x2a66a9272032e8b4,108811841677033779938,4,1579160064781
1124368,0x53349429523d615b:0x2a66a9272032e8b4,102562112468012083826,4,1531282816811
1124369,0x53349429523d615b:0x2a66a9272032e8b4,109274418415646507612,3,1513015611029


We dont have rating text here or the geomap so we will need to transform the JSON to get some of that but it looks like the **GOLD** is in the meta data from the JSON file since this is the **ONLY CSV**

The GMAP_ID is the ID of the business itself so we could join tables pretty easy if we needed to on that but would need to **Group by the business** on some bound so we dont repeat the business and increase the size of the table.

In [14]:
wyoming_metadata#contains business_id, review description, loccation, secotr/category, average rating, num reviews, hours
#Relative_reviews?, and the googlemap_link

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Pawtastic Cuts Mobile Grooming,None,0x89e653296293b521:0x37125d0e7cf86bb2,None,42.424845,-80.443797,"[Pet groomer, Service establishment]",3.4,5,None,None,None,None,"[0x89e7ac97e6710b8d:0xb6619e3c29d7986d, 0x89e6...",https://www.google.com/maps/place//data=!4m2!3...
1,Better Teams,None,0x88f5b912c5e2860f:0x40c4c23bade0547,None,14.158523,-128.960280,"[Business to business service, Service establi...",5.0,8,None,"[[Thursday, 8AM–5PM], [Friday, 8AM–5PM], [Satu...","{'Service options': ['Online appointments'], '...",Closed ⋅ Opens 8AM,None,https://www.google.com/maps/place//data=!4m2!3...
2,Fast Funds Lawsuit Cash Advances,None,0x88d9c7086963457b:0xd27ecdbe17ff2706,None,45.597767,-70.409114,"[Loan agency, Service establishment]",5.0,8,None,"[[Wednesday, 8AM–9PM], [Thursday, 8AM–9PM], [F...","{'Service options': ['Online appointments'], '...",Closed ⋅ Opens 8AM Thu,"[0x88d8df1d6288fc25:0x734897f622c82b27, 0x88d9...",https://www.google.com/maps/place//data=!4m2!3...
3,Naknek Family Fisheries,None,0x56e7db2ef389d417:0x7f05543be7b6696a,None,48.434936,-133.933761,"[Seafood wholesaler, Distribution service, Ser...",5.0,1,None,"[[Wednesday, Closed], [Thursday, Closed], [Fri...",None,Closed ⋅ Opens 7AM Fri,"[0x56e7c54e8354461d:0xf7a08d4c5c020918, 0x56e7...",https://www.google.com/maps/place//data=!4m2!3...
4,"Alternative Offroad, Inc",None,0x80c29dd709d02059:0x6e7d1dfbe9bd3293,None,51.005464,179.431927,"[ATV dealer, Service establishment]",4.5,18,None,"[[Wednesday, 9AM–5PM], [Thursday, 9AM–5PM], [F...",None,Closed ⋅ Opens 9AM Thu,"[0x80c2ea0d1c2c0001:0xd68133ab98377301, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,Summit Hosting,None,0x88f59e7bf89b527d:0x91a7e5cd1c53b55b,None,51.005464,-120.094099,[Web hosting company],4.3,498,None,"[[Friday, Open 24 hours], [Saturday, Open 24 h...",{'Service options': ['Online appointments']},NaN,"[0x88f59e15bd618c59:0xc4c7aafeee09e4c0, 0x88f5...",https://www.google.com/maps/place//data=!4m2!3...
1220,Center Source,None,0x89c688ccf208483b:0x4c51a6ec252f5bb6,None,51.005464,-59.231373,"[Call center, BPO company, Consumer advice cen...",5.0,13,None,"[[Monday, Open 24 hours], [Tuesday, Open 24 ho...",None,NaN,"[0x89c6ed5006ac9d07:0xcb43ee30e4ece8d6, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...
1221,Polar Cooling,None,0x80d6f58e37786355:0x79dfb59cacf74a56,None,31.448221,-114.430913,"[Air conditioning contractor, Air conditioning...",4.6,58,None,"[[Sunday, Closed], [Monday, 7AM–5PM], [Tuesday...",None,NaN,"[0x80d6f5fc749c3105:0xc12e35ff6addd4ac, 0x80d6...",https://www.google.com/maps/place//data=!4m2!3...
1222,Flamingo Florida Kosher Luxury Villas,None,0x88d9add4609c63cd:0xb47ab2ff3db60707,None,51.064604,0.000000,"[Vacation home rental agency, Auction house, G...",5.0,197,None,None,{'Accessibility': ['Wheelchair accessible entr...,NaN,"[0x88d9adcd91f20e23:0x5032a15a9487292c, 0x88d9...",https://www.google.com/maps/place//data=!4m2!3...


Did the state column load in properly?